# Flask Server 
- Summarizing the resume content and Screening Question generation
- Will be integrated with shortlister first.
---
## Importing Dependencies:

In [50]:
import os, re, json, requests, shutil
import numpy as np, pandas as pd
import openai
from langchain.document_loaders import PyMuPDFLoader
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

from flask import Flask, request, jsonify
from dotenv import load_dotenv
from flask_cors import CORS
import sqlite3

## OpenAI api key fetching:

In [2]:
load_dotenv()

True

In [3]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

---

## Document Loader:

In [4]:
path_to_cv = os.path.join('samples_resume',os.listdir('samples_resume')[0])
path_to_cv

'samples_resume\\00.pdf'

In [5]:
def load_cv(path_to_cv):
    loader = PyMuPDFLoader(path_to_cv)
    document = loader.load()
    return document

In [6]:
# load_cv(path_to_cv)

In [7]:
# os.remove(os.path.join('tmp',os.listdir('tmp')[0]))

In [8]:
# os.listdir('tmp')[0]

In [9]:
# shutil.rmtree(os.path.join('tmp',os.listdir('tmp')[0]))

---
## Embeddings setup and persist_directory path setup for chromaDB:

In [48]:
embeddings = OpenAIEmbeddings()
persist_dir = os.path.join('app_persist_directory')
llm = ChatOpenAI()

---

### Open AI test:

In [49]:
prompts = ["What job role is suitable for this resume? [only job role word: No explanation]", 
         "should I shortlist this resume for the {} job role? [only yes|no answer: no explanation]",
         "top technical skills -list [only wordings separated by commas, no explanation] in given resume",
          "Other possible job role candidate is eligible for? [list separated by commas, only words: no explanation",
         "On the basis of the {} skillsets, generate {} logical based hard level screening questions [hard-level : MCQs], Also provide answers at the very end of the output separated by commas",
          "Summarize this resume in {} words"         
         ]

In [51]:
DATABASE_path = os.path.join('database', 'raw_database.db')
DATABASE_path

'database\\raw_database.db'

In [45]:
path_to_cv = os.path.join('samples_resume',os.listdir('samples_resume')[0])
path_to_cv

'samples_resume\\00.pdf'

---

## Flask Server Building:

In [54]:
app = Flask(__name__)
CORS(app)

@app.route('/')
def hello():
    print("http://127.0.0.1:5000/")
    return "server is working..."


@app.route('/test', methods=['POST'])
def test():
    data = request.get_json()
    # print(type(data))
    if data is not None:
        return jsonify({"Status": "working"})
    return jsonify({"Status": "No data recieved"})

@app.route('/summarize', methods=['POST'])
def summarize():
    # Fetching Details
    data = request.get_json()
    
    name = data['name']
    contact_no = data['contact_no']
    email = data['email']
    skills = data['skills']
    projectLinks = data['projectLinks']
    job_role = data['job_role']
    # resume_file = data['resume_file']
    
    # eligible = data['eligibility']
    # job_role_predicted = data['job_role_predicted']
    # skills_extracted = data['skills_extracted']

    resume_file = 'Temp_Test'
    loader = PyMuPDFLoader(path_to_cv)
    document = loader.load()

    vectordb = Chroma.from_documents(
        documents = document,
        embedding = embeddings,
        persist_directory = persist_dir
    )
    vectordb.persist()
    


    retriever = vectordb.as_retriever(search_kwargs={'k':3})
    qa = RetrievalQA.from_chain_type(llm=llm, chain_type='stuff', retriever=retriever)

    query = f"###Prompt {prompts[0]}"
    llm_response = qa(query)
    job_title_prediction = llm_response['result']

    query = f"###Prompt {prompts[1].format(job_role)}"
    llm_response = qa(query)
    isEligible = llm_response['result']

    query = f"###Prompt {prompts[2]}"
    llm_response = qa(query)
    skills_extracted = llm_response['result']

    query = f"###Prompt {prompts[3]}"
    llm_response = qa(query)
    job_role_predicted = llm_response['result']

    query = f"###Prompt {prompts[4].format(skills_extracted, 15)}"
    llm_response = qa(query)
    screening_questions = llm_response['result']

    query = f"###Prompt {prompts[5].format(60)}"
    llm_response = qa(query)
    summary = llm_response['result']


    # More evaluation steps here:




    # Pushing into Database
    conn = sqlite3.connect(DATABASE_path)
    cur = conn.cursor()

    cur.execute(f"INSERT INTO table_name...")


    conn.commit()
    conn.close()
    vectordb.delete_collection()
    
    

    return jsonify({"message": "Congratulations You have applied for the following role successfully!!"})


    
    

In [55]:
if __name__ == "__main__":
    # app.run(debug=True)
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [10/Aug/2023 13:00:38] "POST /add HTTP/1.1" 404 -
127.0.0.1 - - [10/Aug/2023 13:01:03] "GET /add HTTP/1.1" 404 -
127.0.0.1 - - [10/Aug/2023 13:01:08] "GET / HTTP/1.1" 200 -


http://127.0.0.1:5000/


127.0.0.1 - - [10/Aug/2023 13:09:20] "POST /add HTTP/1.1" 404 -
127.0.0.1 - - [10/Aug/2023 13:09:22] "POST /add HTTP/1.1" 404 -
127.0.0.1 - - [10/Aug/2023 13:09:26] "GET /add HTTP/1.1" 404 -
